## Event Detection

In [ ]:
import pandas as pd

In [ ]:
#We obtain the events, transform to the date format and filter for the year 2019
data_raw = pd.read_csv('Events_BBDD.csv')
data_raw['AIS_UPDATE_DATE'] = pd.to_datetime(data_raw['AIS_UPDATE_DATE'], format = '%Y-%m-%d %H:%M:%S')
data_raw = data_raw[data_raw['AIS_UPDATE_DATE'].dt.year == 2019]
data = data_raw.copy(deep = True)

In [ ]:
#We filter for only events of interest 6: Pilotage Start, 7: Towing Start, 27: Excessive Speed
data = data[data['ID_EVENT_TYPE'].isin([6,7,27])]
data = data[(data['AIS_LAT'] < 90) & (data['AIS_LAT'] > -90) ]
data = data[(data['AIS_LON'] < 180) & (data['AIS_LON'] > -180)]
data = data.dropna(subset = ['AIS_LAT', 'AIS_LON'])
data.head(5)

In [ ]:
import gmaps
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html

#For each of the events, we make heat maps to see the areas where they are usually detected
gmaps.configure(api_key='')
temp_6 = pd.DataFrame()
temp_6['latitude'] = data[data['ID_EVENT_TYPE'] == 6]['AIS_LAT']
temp_6['longitude'] = data[data['ID_EVENT_TYPE'] == 6]['AIS_LON']
fig = gmaps.figure(map_type='TERRAIN', display_toolbar = True)
heatmap_layer = gmaps.heatmap_layer(temp_6)
fig.add_layer(heatmap_layer)
embed_minimal_html('PilotageStartsArea.html', views=[fig])

In [ ]:
gmaps.configure(api_key='')
temp_7 = pd.DataFrame()
temp_7  ['latitude'] = data[data['ID_EVENT_TYPE'] == 7]['AIS_LAT']
temp_7['longitude'] = data[data['ID_EVENT_TYPE'] == 7]['AIS_LON']
fig = gmaps.figure(map_type='ROADMAP', display_toolbar = True)
heatmap_layer = gmaps.heatmap_layer(temp_7)
fig.add_layer(heatmap_layer)
embed_minimal_html('TowingStarsArea.html', views=[fig])

In [ ]:
gmaps.configure(api_key='')
temp_27 = pd.DataFrame()
temp_27['latitude'] = data[data['ID_EVENT_TYPE'] == 27]['AIS_LAT']
temp_27['longitude'] = data[data['ID_EVENT_TYPE'] == 27]['AIS_LON']
fig = gmaps.figure(map_type='ROADMAP', display_toolbar = True)
heatmap_layer = gmaps.heatmap_layer(temp_27)
fig.add_layer(heatmap_layer)
embed_minimal_html('SpeedExceedAreas.html', views=[fig])

In [ ]:
import seaborn as sns 
#We represent the events detected for each of the months in a bar plot.
df_plot = pd.DataFrame()
df_plot['event'] = data_raw['ID_EVENT_TYPE']
df_plot['integer'] = 1
df_plot['event'] = data_raw['ID_EVENT_TYPE']
df_plot['time'] = data_raw['AIS_UPDATE_DATE']
dict_events = {'others' : 'Others Events', 6 : 'Pilotage Start', 7: 'Towage Start', 27: 'Speed Exceed'}
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
month_index = dict(zip(months, range(len(months))))

df_plot['event_desc'] = df_plot['event'].map(dict_events)
df_plot['month']= df_plot['time'].apply(lambda x: x.month_name())
df_plot = df_plot.groupby(['event_desc', 'month'])['integer'].sum()
df_plot = df_plot.reset_index()
df_plot['month_rank'] = df_plot['month'].map(month_index)
df_plot = df_plot.sort_values(by =['month_rank'])

g = sns.catplot(data = df_plot, x='month', y = 'integer', hue = 'event_desc', kind = 'bar')
g.set_xticklabels(rotation = 45)
g.set(xlabel='Month', ylabel='Number of events detected')
g._legend.set_title('Event')
g.savefig('BarPlotNumberEvents.png', dpi = 800, bbox_inches = 'tight') 

In [ ]:
#Calculate length and beam attributes
data_raw = data_raw[data_raw['ID_EVENT_TYPE'].isin([6,7,27])]
data_raw['length'] = data_raw['AIS_TO_BOW'] + data_raw['AIS_TO_STERN']
data_raw['width'] = data_raw['AIS_TO_PORT'] + data_raw['AIS_TO_STARBOARD']

In [ ]:
#We represent the characteristics calculated for each of the events through a scatter plot with distribution of the variables in the axes.
g6 = sns.jointplot("length", "width", data = data_raw[data_raw['ID_EVENT_TYPE'] == 6], kind = 'reg')
g6.fig.suptitle("Pilotage Ship Specs")
g6.fig.tight_layout()
g6.set_axis_labels("Length (m)", "Width (m)")
g6.fig.subplots_adjust(top=0.95)
g6.savefig('PilotageShipSpecs.png')

In [ ]:
g7 = sns.jointplot("length", "width", data = data_raw[data_raw['ID_EVENT_TYPE'] == 7], kind = 'reg')
g7.fig.suptitle("Towage Ship Specs")
g7.fig.tight_layout()
g7.set_axis_labels("Length (m)", "Width (m)")
g7.fig.subplots_adjust(top=0.95)
g7.savefig("TowageShipSpecs.png")

In [ ]:
g27 = sns.jointplot("length", "width", data = data_raw[data_raw['ID_EVENT_TYPE'] == 27], kind = 'reg')
g27.fig.suptitle("Speeding Ship Specs")
g27.fig.tight_layout()
g27.set_axis_labels("Length (m)", "Width (m)")
g27.fig.subplots_adjust(top=0.95)
g27.savefig("SpeedingSpeds.png")

In [ ]:
#In the following graph we represent the properties of the three events to see the differences between them.
data_raw['event_desc'] = data_raw['ID_EVENT_TYPE'].map(dict_events)
grid = sns.JointGrid(x = 'length', y = 'width', data = data_raw)
g = grid.plot_joint(sns.scatterplot, hue = 'event_desc', data = data_raw)

sns.kdeplot(data_raw[data_raw['event_desc'] == 'Pilotage Start']['length'], ax = g.ax_marg_x, legend = False)
sns.kdeplot(data_raw[data_raw['event_desc'] == 'Towage Start']['length'], ax = g.ax_marg_x, legend = False)
sns.kdeplot(data_raw[data_raw['event_desc'] == 'Speed Exceed']['length'], ax = g.ax_marg_x, legend = False)

sns.kdeplot(data_raw[data_raw['event_desc'] == 'Pilotage Start']['width'], ax = g.ax_marg_y, legend = False)
sns.kdeplot(data_raw[data_raw['event_desc'] == 'Towage Start']['width'], ax = g.ax_marg_y, legend = False)
sns.kdeplot(data_raw[data_raw['event_desc'] == 'Speed Exceed']['width'], ax = g.ax_marg_y, legend = False)
g.savefig("ScatterUB_Events.png")

In [ ]:
# MMSI AND SHIP TYPE ARE CATEGORICAL VARIABLES
data_clean['MMSI'] = data_clean['MMSI'].astype(str)
data_clean['SHIP_TYPE'] = data_clean['SHIP_TYPE'].astype(str)
data_clean

In [ ]:
#We make a firts approach using Random Forest
features = data_clean[['LON', 'LAT', 'LENGTH', 'WIDTH', 'SPEED', 'MMSI', 'SHIP_TYPE']]
labels = data_clean['EVENT_DESC']

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, stratify = labels, test_size = 0.25)
forest = RandomForestClassifier(n_estimators=25, criterion = 'gini', random_state= 1, n_jobs= -1)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'weighted'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'weighted'))
print("recall: ", recall_score(y_test, y_pred, average = 'weighted'))

import numpy as np
import matplotlib.pyplot as plt

feat_labels = features.columns
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" %(f+1, 30, feat_labels[indices[f]], importances[indices[f]])

#We represent a bar plot with features importances given by Random Forest
plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), importances[indices], align = 'center')
plt.xticks(range(X_train.shape[1]), feat_labels[indices], rotation = -45)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.savefig("FeaturesImportance.png")
plt.ylabel('Gini Importance Values')
plt.xlabel('Features')
plt.savefig('FeaturesImportance.png', dpi = 800, bbox_inches = 'tight' )
plt.show()


In [ ]:
#We repeat the process removing SHIP_TYPE and MMSI categories.
features = data_clean[['LON', 'LAT', 'LENGTH', 'WIDTH', 'SPEED']]
labels = data_clean['EVENT_DESC']

X_train, X_test, y_train, y_test = train_test_split(features, labels, stratify = labels, test_size = 0.25)
forest = RandomForestClassifier(n_estimators=25, criterion = 'gini', random_state= 1, n_jobs= -1)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'weighted'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'weighted'))
print("recall: ", recall_score(y_test, y_pred, average = 'weighted'))

feat_labels = features.columns
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" %(f+1, 30, feat_labels[indices[f]], importances[indices[f]]))

plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), importances[indices], align = 'center')
plt.xticks(range(X_train.shape[1]), feat_labels[indices], rotation = -45)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()

In [ ]:
#We use sequential feature selection is used in order to see which features make the most accuracy prediction
from sklearn.neighbors import KNeighborsRegressor
from varios import SBS

sbs = SBS(forest, k_features = 2)
sbs.fit(features, labels)

In [ ]:
#Bar plot with scores of each combinatios computed
k_feat = [len(k) for k in sbs.subsets_]
plt.plot(k_feat, sbs.scores_, marker = 'o')
plt.ylim([0, 1])
plt.ylabel('Accuracy')
plt.xlabel('Number of features')
plt.grid()
plt.tight_layout()
plt.savefig("LinePlot_DroppingFeatures.png")
plt.show()

In [ ]:
#Combination with most accuracy
indices = list(sbs.subsets_[1])
features.columns[indices]

In [ ]:
#Now Principal Component Analysis is performed to see which directions explain the most variance of the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
features_sc = sc.fit_transform(features)

cov_mat = np.cov(features_sc.T)
eigen_values, eigen_vectors = np.linalg.eig(cov_mat)
tot = sum(eigen_values)
var_exp = [(i / tot) for i in sorted(eigen_values, reverse = True)]
cum_var_exp = np.cumsum(var_exp)
plt.bar(range(1,6), var_exp, alpha = 0.5, align = 'center', label = 'Individual explained variance')
plt.step(range(1, 6), cum_var_exp, where= 'mid', label = 'Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc = 'best')
plt.tight_layout()
plt.savefig("ExplainedVariance.png", dpi = 800, bbox_inches = 'tight' )

In [ ]:
eigen_pairs = [(np.abs(eigen_values[i]), eigen_vectors[:, i]) for i in range(len(eigen_values))]
eigen_pairs.sort(key = lambda k: k[0], reverse = True)
w = np.hstack((eigen_pairs[0][1][:, np.newaxis], eigen_pairs[1][1][:, np.newaxis], eigen_pairs[2][1][:, np.newaxis]))

In [ ]:
features_pca = features_sc.dot(w)
features_pca

In [ ]:
#Scatter plot of data points taken into this new dimensional space
colors = ['r', 'b', 'g']
markers = ['s', 'x', 'o']

for l, c, m in zip (np.unique(labels), colors, markers):
    plt.scatter(features_pca[labels == l][0], features_pca[labels == l][0], c=c, label = l, marker = m)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc = 'upper left')
plt.tight_layout()
plt.show()

In [ ]:
features_pca[labels == np.unique(labels)[2]]

In [ ]:
#Random Forest is applied to see if PCA improved the accuracy
features_pca = pd.DataFrame(features_pca)

X_train, X_test, y_train, y_test = train_test_split(features_pca, labels, stratify = labels, test_size = 0.25)
forest = RandomForestClassifier(n_estimators=25, criterion = 'gini', random_state= 1, n_jobs= -1)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'weighted'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'weighted'))
print("recall: ", recall_score(y_test, y_pred, average = 'weighted'))


feat_labels = features_pca.columns
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" %(f+1, 30, feat_labels[indices[f]], importances[indices[f]]))

plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), importances[indices], align = 'center')
plt.xticks(range(X_train.shape[1]), feat_labels[indices], rotation = -45)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.savefig('Feature_ImportancesPCA.png')
plt.show()

In [ ]:
#A new feature extraction technique is used, where directions which maximizes class separability are the new objective
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components= 3)
X_train, X_test, y_train, y_test = train_test_split(features_sc, labels, stratify = labels, test_size = 0.25)
X_train = lda.fit_transform(X_train, y_train)

forest = RandomForestClassifier(n_estimators=25, criterion = 'gini', random_state= 1, n_jobs= -1)
forest.fit(X_train, y_train)
X_test = lda.transform(X_test)
y_pred = forest.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'weighted'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'weighted'))
print("recall: ", recall_score(y_test, y_pred, average = 'weighted'))

In [ ]:
#Instead of PCA, we try Kernel Principal Component Analysis with Radial Basis Function Kernel, if there is some relation non linearly that was not seen.
from sklearn.decomposition import KernelPCA
scikit_kpca = KernelPCA(n_components = 2, kernel = 'rbf', gamma = 15 )
X_train, X_test, y_train, y_test = train_test_split(features_sc, labels, stratify = labels, test_size = 0.25)
X_train = scikit_kpca.fit_transform(X_train, y_train)

forest = RandomForestClassifier(n_estimators=25, criterion = 'gini', random_state= 1, n_jobs= -1)
forest.fit(X_train, y_train)
X_test = scikit_kpca.transform(X_test)
y_pred = forest.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'weighted'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'weighted'))
print("recall: ", recall_score(y_test, y_pred, average = 'weighted'))

In [ ]:
#We repeat the full process but using K-Nearest-Neighbots
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(features_sc, labels, stratify = labels, test_size = 0.25)
knn = KNeighborsClassifier(n_neighbors = 5, p = 2, metric = 'minkowski')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'weighted'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'weighted'))
print("recall: ", recall_score(y_test, y_pred, average = 'weighted'))

lda = LDA(n_components= 3)
X_train, X_test, y_train, y_test = train_test_split(features_sc, labels, stratify = labels, test_size = 0.25)
X_train = lda.fit_transform(X_train, y_train)

knn = KNeighborsClassifier(n_neighbors = 5, p = 2, metric = 'minkowski')
knn.fit(X_train, y_train)
X_test = lda.transform(X_test)
y_pred = knn.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'weighted'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'weighted'))
print("recall: ", recall_score(y_test, y_pred, average = 'weighted'))

scikit_kpca = KernelPCA(n_components = 2, kernel = 'rbf', gamma = 15 )
X_train, X_test, y_train, y_test = train_test_split(features_sc, labels, stratify = labels, test_size = 0.25)
X_train = scikit_kpca.fit_transform(X_train, y_train)


knn.fit(X_train, y_train)
X_test = scikit_kpca.transform(X_test)
y_pred = knn.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'weighted'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'weighted'))
print("recall: ", recall_score(y_test, y_pred, average = 'weighted'))

pca = PCA(n_components= 2)
X_train, X_test, y_train, y_test = train_test_split(features_sc, labels, stratify = labels, test_size = 0.25)
X_train = pca.fit_transform(X_train, y_train)


knn.fit(X_train, y_train)
X_test = pca.transform(X_test)
y_pred = knn.predict(X_test)

print("Report: ", classification_report(y_test, y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred, average = 'macro'))
print("precision_score: ", precision_score(y_test, y_pred, average = 'macro'))
print("recall: ", recall_score(y_test, y_pred, average = 'macro'))

In [ ]:
#Now we perfom prediction of an event given the sequence of AIS Messages. 
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

data_raw = pd.read_csv('C:/Users/sergi/Desktop/Pixel/CEP/Events_BBDD.csv')
data_raw['AIS_UPDATE_DATE'] = pd.to_datetime(data_raw['AIS_UPDATE_DATE'], format = '%Y-%m-%d %H:%M:%S')
data_raw = data_raw[data_raw['AIS_UPDATE_DATE'] > '2019-07-15']
data = data_raw.copy(deep = True)
data = data[data['ID_EVENT_TYPE'].isin([6,7,27])]
data = data[(data['AIS_LAT'] < 90) & (data['AIS_LAT'] > -90) ]
data = data[(data['AIS_LON'] < 180) & (data['AIS_LON'] > -180)]
#data = data.dropna(subset = ['AIS_LAT', 'AIS_LON'])
dict_events = {'others' : 'Others Events', 6 : 'Pilotage Start', 7: 'Towage Start', 27: 'Speed Exced'}
data['ID_EVENT_TYPE'] = data['ID_EVENT_TYPE'].map(dict_events)
#data.head()

# All the AIS messages from the logs are retrieved
df = pd.read_csv('C:/Users/sergi/Desktop/Pixel/CEP/LogsGrouped.csv')

# AIS messages are filtered to only retrieve ones with valid locations
df = df[(abs(df['lat']) < 90) & (abs(df['lon']) < 180)]
df['updateAsDate'] = pd.to_datetime(df['updateAsDate'], format='%d/%m/%Y %H:%M:%S')
df

def feature_eng(events, df, rows):
    labels_array = []
    features_array = []
    i = 0
    for index, row in events.iterrows():

        label = row['ID_EVENT_TYPE']
        df_temp = pd.DataFrame()
        mmsi = row['AIS_MMSI']
        time = row['AIS_UPDATE_DATE']
        df_temp = df[df['mmsi'] == mmsi]
        df_temp = df_temp[df_temp['updateAsDate'] <= time]
        df_temp.sort_values(by = ['updateAsDate'], inplace = True, ascending = False)
        df_temp = df_temp.head(rows)
        
        if df_temp.shape[0]==30:
            df_temp['length'] = df_temp['to_bow'] + df_temp['to_stern']
            df_temp['width'] = df_temp['to_port'] + df_temp['to_starboard']
            df_temp['Av.Speed'] = df_temp['speed'].expanding().mean()
            df_temp = df_temp[['lat', 'lon', 'speed', 'length', 'width', 'course', 'turn', 'draught', 'heading' ]]
            scaler = StandardScaler()
            df_temp = scaler.fit_transform(df_temp)
            features_array.append(df_temp.tolist())
            labels_array.append(label)
            i = i + 1

    return features_array, labels_array, df_temp


features_array, labels_arraty, prueba = feature_eng(data, df, 30)

labels_array = pd.DataFrame(labels_arraty, columns = ['ID_EVENT'])
labels_array = pd.get_dummies(labels_array, columns = ['ID_EVENT'])
labels_array.values.tolist()

features_array = np.asarray(features_array)
labels_array = np.asarray(labels_array)

X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size = 0.25)

model = Sequential()
model.add(LSTM(32, input_shape = (30,9)))
model.add(Dense(3, activation = 'sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.savefig("C:/Users/sergi/Desktop/Pixel/CEP/Loss.png")

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.savefig("C:/Users/sergi/Desktop/Pixel/CEP/Accuracy.png")
plt.legend()